# Unet Pytorch for Inferentia (NeuronSDK)
In this example, you'll see how to compile a public implementation of an Unet Model created for Pytorch to Inferentia (inf1 instances)

In [ ]:
# if necessary
!pip install neuron-cc[torch] 

### Get Unet for Pytorch from Github
This is a public implementation of the model

In [2]:
!git clone https://github.com/jaxony/unet-pytorch unet

Cloning into 'unet'...
remote: Enumerating objects: 28, done.
remote: Total 28 (delta 0), reused 0 (delta 0), pack-reused 28
Receiving objects: 100% (28/28), 42.01 KiB | 2.33 MiB/s, done.
Resolving deltas: 100% (12/12), done.


### Import common libraries

In [22]:
import sys
sys.path.insert(0, 'unet')
import torch
import numpy as np
from model import UNet
from torch.autograd import Variable
torch.__version__

'1.10.2+cu102'

### Intialize a model with random weights
--> You need to train the model first to get something useful. In this example we're just testing compatibility + performance

In [23]:
model = UNet(3, depth=5, merge_mode='concat').eval()
x = torch.rand(1, 3, 320, 320)
_ = model(x)

unet/model.py:211: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
unet/model.py:212: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


### Simple benchmark on CPU

In [24]:
# test on CPU
%timeit model(x)

238 ms ± 8.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Tracing the model with NeuronSDK

In [17]:
import torch.neuron
neuron_model = torch.neuron.trace(
    model,
    x,
    verbose='debug'
)

INFO:Neuron:All operators are compiled by neuron-cc (this does not guarantee that neuron-cc will successfully compile)
INFO:Neuron:Number of arithmetic operators (pre-compilation) before = 49, fused = 49, percent fused = 100.0%
INFO:Neuron:Number of neuron graph operations 577 did not match traced graph 569 - using heuristic matching of hierarchical information
INFO:Neuron:Compiling function _NeuronGraph$118 with neuron-cc
INFO:Neuron:Compiling with command line: '/home/ec2-user/anaconda3/envs/aws_neuron_pytorch_p36/bin/neuron-cc compile /tmp/tmpwx2yvtn7/graph_def.pb --framework TENSORFLOW --pipeline compile SaveTemps --output /tmp/tmpwx2yvtn7/graph_def.neff --io-config {"inputs": {"0:0": [[1, 3, 320, 320], "float32"]}, "outputs": ["Conv2d_32/aten__convolution/transpose_2:0"]} --verbose debug'
INFO:Neuron:skip_inference_context for tensorboard symbols at /home/ec2-user/anaconda3/envs/aws_neuron_pytorch_p36/lib/python3.6/site-packages/torch_neuron/tensorboard.py:305 tb_parse
INFO:Neuron

In [18]:
neuron_model.save('unet.pt')

### Load the model from disk to see if it worked

In [26]:
neuron_model = torch.load('unet.pt')
_=neuron_model.eval()
# warmup
_=neuron_model(x)

### Simple benchmark on 1 core of the Inf1 instance

In [27]:
%timeit neuron_model(x)

16.5 ms ± 72.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
